In [0]:
# Databricks notebook source
# ============================================================
# UNITY CATALOG SANITY CHECK — UK Flood Monitoring Project
# ============================================================
# This notebook validates that Unity Catalog and associated
# schemas, volumes, and privileges are correctly configured.
# ============================================================

print("Running Unity Catalog sanity check...")


Running Unity Catalog sanity check...


In [0]:
# Confirm user identity and default catalog context
display(spark.sql("SELECT current_user() AS user, current_catalog() AS catalog, current_schema() AS schema"))

user,catalog,schema
sheldon.kemper@outlook.com,hive_metastore,default


In [0]:
# List all catalogs visible to your user
print("=== Catalogs ===")
display(spark.sql("SHOW CATALOGS"))

=== Catalogs ===


catalog
dbw_flood_dev_3577787633845022
flood_dev
hive_metastore
samples
system


In [0]:
# Use the flood_dev catalog and list schemas
spark.sql("USE CATALOG flood_dev")
print("=== Schemas in flood_dev ===")
display(spark.sql("SHOW SCHEMAS IN flood_dev"))

=== Schemas in flood_dev ===


databaseName
bronze
gold
information_schema
silver


In [0]:
# List objects in each schema
for schema in ["bronze", "silver", "gold"]:
    print(f"=== {schema.upper()} Tables ===")
    display(
        spark.sql(
            f"SHOW TABLES IN flood_dev.{schema}"
        )
    )

=== BRONZE Tables ===


database,tableName,isTemporary


=== SILVER Tables ===


database,tableName,isTemporary


=== GOLD Tables ===


database,tableName,isTemporary


In [0]:
# List available volumes (for raw and checkpoint)
print("=== Volumes in flood_dev.bronze ===")
display(spark.sql("SHOW VOLUMES IN flood_dev.bronze"))

=== Volumes in flood_dev.bronze ===


database,volume_name
bronze,checkpoints
bronze,raw_payloads


In [0]:
# Test table creation (should succeed only where you have write access)
try:
    spark.sql("""
        CREATE OR REPLACE TABLE flood_dev.bronze._uc_sanity_test (
            id INT,
            note STRING
        ) USING DELTA
    """)
    spark.sql("INSERT INTO flood_dev.bronze._uc_sanity_test VALUES (1, 'Terraform verified')")
    display(spark.sql("SELECT * FROM flood_dev.bronze._uc_sanity_test"))
    spark.sql("DROP TABLE flood_dev.bronze._uc_sanity_test")
    print("✅ Write permissions verified in bronze schema")
except Exception as e:
    print(f"⚠️ Write test failed (expected if read-only): {e}")

id,note
1,Terraform verified


✅ Write permissions verified in bronze schema


In [0]:
# Inspect volume definitions
for vol in ["checkpoints", "raw_payloads"]:
    print(f"=== Volume: {vol} ===")
    display(spark.sql(f"DESCRIBE VOLUME flood_dev.bronze.{vol}"))

=== Volume: checkpoints ===


name,catalog,database,owner,storage_location,volume_type,comment,securable_type,securable_kind
checkpoints,flood_dev,bronze,sheldon.kemper@outlook.com,abfss://uc-flood-dev@flooducuksouthdev.dfs.core.windows.net/__unitystorage/catalogs/bb05b4b6-3d86-4de9-ad59-476f7d4b564d/volumes/d13b768f-7793-4a02-85fb-116c3551c50c,MANAGED,DLT checkpoint storage for streaming pipelines,VOLUME,VOLUME_STANDARD


=== Volume: raw_payloads ===


name,catalog,database,owner,storage_location,volume_type,comment,securable_type,securable_kind
raw_payloads,flood_dev,bronze,sheldon.kemper@outlook.com,abfss://uc-flood-dev@flooducuksouthdev.dfs.core.windows.net/__unitystorage/catalogs/bb05b4b6-3d86-4de9-ad59-476f7d4b564d/volumes/99a8b67c-64d3-4002-83b3-fc7a8c483fc3,MANAGED,Raw API payloads from UK Flood Monitoring API,VOLUME,VOLUME_STANDARD


In [0]:

print("✅ Sanity check complete — Unity Catalog verified successfully.")


✅ Sanity check complete — Unity Catalog verified successfully.
